In [1]:
import pandas as pd
import SingleFlash_125_Features
import time
import numpy as np
import math

import datetime

In [2]:
tit="N正常.csv"
tit="N异常.csv"
tit="P正常.csv"
tit="P异常.csv"
r="ALL正常.csv"
l="ALL异常.csv"

In [3]:
path = 'E:\\desktop\\data\\'
def soso(tit):
    #读取数据
    try:
        data = pd.read_csv(path+tit,encoding='gbk')
    except:
        data = pd.read_csv(path+tit,encoding='utf-8')

    # name=pd.DataFrame(list(data.iloc[:,0]),columns=["name"])
    name_list=list(data.iloc[:,0])
    try:
        # 对瞳孔半径列进行分割
        df=data['瞳孔半径'].astype('str').str.split(',',expand=True)
        # 修改分割后的字段名称
        df.columns=[str(i) for i in df.columns]
    except:
        df=pd.DataFrame(data.iloc[:,1:126])

    df=df.astype('float')
    exception=0
    fea_dict={}
    for i in range(len(df)):#
        row=list(df.iloc[i]) #每一行数据
        fea=SingleFlash_125_Features.Features(row)

        if fea.exception:
            exception+=1
        row_dict=fea.judge_exception()
        for key in row_dict.keys():
            if len(fea_dict)!=len(row_dict)+1:
                fea_dict['name']=[]
                fea_dict[key]=[]

            fea_dict[key].append(row_dict.get(key))
        fea_dict['name'].append(name_list[i])
    fea_df=pd.DataFrame(fea_dict)
    fea_df = fea_df.set_index('name')
    print(exception/len(fea_df))
    return fea_df

In [4]:
def get_crit(n_fea, p_fea, fea_list):
    crit = {}
    for fea in fea_list:
        n_col = list(n_fea[fea])
        p_col = list(p_fea[fea])
        np_col = n_col+p_col
        s_dict = {'s': [], 'value': [], 'plou': []}
        for i in range(len(np_col)):
            s = np.sum(np.array(n_col) -
                        np_col[i])+np.sum(np.array(p_col)-np_col[i])
            s_dict['s'].append(abs(s))
            s_dict['value'].append(np_col[i])
            if np.sum(np.array(p_col)-np_col[i]) >= 0:
                s_dict['plou'].append('up')
            else:
                s_dict['plou'].append('down')
        minindex = s_dict['s'].index(min(s_dict['s']))
        crit[fea] = [s_dict['value'][minindex], s_dict['plou'][minindex]]

    # 寻找阴阳聚集点
    fix_crit={}
    for fea in fea_list:
        n_col = list(n_fea[fea])
        p_col = list(p_fea[fea])
        if crit[fea][1] == "up":
            n_col.sort(reverse=False)  # 正序
            fix_crit[fea]=n_col[len(n_col)*97//100] # 阳性聚集点
        else:
            n_col.sort(reverse=True)  # 正序
            fix_crit[fea]=n_col[len(n_col)*97//100]  # 阳性聚集点
    return fix_crit

In [5]:
r_fea=soso(r)
l_fea=soso(l)
fea_list=list(r_fea.columns)

get_crit(r_fea, l_fea, fea_list)

g:\work\特征算法12.2\SingleFlash_125_Features.py:352: RuntimeWarning: invalid value encountered in double_scalars
  f4 = 2 - (np.std(temp4) / np.mean(temp4)) * 10  # 根据变异系数判断异常值范围
g:\work\特征算法12.2\SingleFlash_125_Features.py:188: RuntimeWarning: invalid value encountered in true_divide
  fixup = 1-(row/(np.abs(ini_row - row)+row))
g:\work\特征算法12.2\SingleFlash_125_Features.py:61: RuntimeWarning: divide by zero encountered in double_scalars
  'B': (x1 - x2) / (y2 - y1),
g:\work\特征算法12.2\SingleFlash_125_Features.py:62: RuntimeWarning: invalid value encountered in double_scalars
  'C': ((x1 - x2) * y1 / (y2 - y1)) - x1
g:\work\特征算法12.2\SingleFlash_125_Features.py:78: RuntimeWarning: invalid value encountered in double_scalars
  dl = abs(A * x0 + B * y0 + C) / pow(pow(A, 2) + pow(B, 2), 0.5)
g:\work\特征算法12.2\SingleFlash_125_Features.py:151: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(abs(ini_row - row)/np.mean(ini_row))


0.06432038834951456


g:\work\特征算法12.2\SingleFlash_125_Features.py:352: RuntimeWarning: invalid value encountered in double_scalars
  f4 = 2 - (np.std(temp4) / np.mean(temp4)) * 10  # 根据变异系数判断异常值范围
g:\work\特征算法12.2\SingleFlash_125_Features.py:61: RuntimeWarning: divide by zero encountered in double_scalars
  'B': (x1 - x2) / (y2 - y1),
g:\work\特征算法12.2\SingleFlash_125_Features.py:62: RuntimeWarning: invalid value encountered in double_scalars
  'C': ((x1 - x2) * y1 / (y2 - y1)) - x1
g:\work\特征算法12.2\SingleFlash_125_Features.py:78: RuntimeWarning: invalid value encountered in double_scalars
  dl = abs(A * x0 + B * y0 + C) / pow(pow(A, 2) + pow(B, 2), 0.5)
g:\work\特征算法12.2\SingleFlash_125_Features.py:188: RuntimeWarning: invalid value encountered in true_divide
  fixup = 1-(row/(np.abs(ini_row - row)+row))
g:\work\特征算法12.2\SingleFlash_125_Features.py:328: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 - (np.std(temp1) / np.mean(temp1)) * 10  # 根据变异系数判断异常值范围
g:\work\特征算法12.2\SingleFlash_1

0.5384615384615384


{'fix_s0': 0.41630177102217353,
 'fix_s1': 0.04293427529626193,
 'fix_s2': 0.17816964809384006,
 'fix_s3': 0.1393172685489167,
 'fix_num0': 31.399999999999938,
 'fix_num1': 8.4,
 'fix_num2': 11.399999999999997,
 'fix_num3': 14.599999999999996,
 'fix_extent_median0': 0.007629600452124463,
 'fix_extent_median1': 0.015873015873015928,
 'fix_extent_median2': 0.01005490196078429,
 'fix_extent_median3': 0.008964346883298502,
 'fix_extent_mean0': 0.02944961172140653,
 'fix_extent_mean1': 0.04843168802260982,
 'fix_extent_mean2': 0.02830448409587059,
 'fix_extent_mean3': 0.037478812762854495,
 'fix_var0': 25.278676731671112,
 'fix_var1': 9.419213279600001,
 'fix_var2': 5.451495381795556,
 'fix_var3': 15.89516533392}

In [6]:
r_fea.to_csv(path+'fea\\'+'[特征]'+r,encoding='gbk')
l_fea.to_csv(path+'fea\\'+'[特征]'+l,encoding='gbk')


print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
print(tit)
print("去异常文件已保存 👌")
print("特征提取完毕 👌")

2023-02-22 20:31:49
P异常.csv
去异常文件已保存 👌
特征提取完毕 👌
